# Informe final - Sistema de Gestión y Recomendación de Viajes  
**Trabajo Práctico Integrador – Bases de Datos NoSQL**  
**Autores:** Dario Micheli y Pablo Luberriaga 
**Fecha:** Octubre 2025 

---

## Introducción  

El presente trabajo integra tres tecnologías NoSQL — **MongoDB**, **Neo4j** y **Redis** — para desarrollar un **Sistema de Gestión y Recomendación de Viajes**.  

El objetivo principal fue modelar distintos tipos de información y operaciones que reflejan un entorno real de gestión turística:
- **MongoDB** para almacenar datos descriptivos y persistentes (usuarios, destinos, hoteles, actividades, reservas).  
- **Neo4j** para modelar relaciones y realizar recomendaciones basadas en vínculos entre usuarios y destinos.  
- **Redis** para manejar información temporal en memoria, como usuarios conectados, búsquedas recientes y reservas no confirmadas.  

El desarrollo se realizó en **Python (JupyterLab)**, integrando las tres bases mediante sus respectivos controladores (`pymongo`, `neo4j-driver`, `redis-py`) y visualizando los resultados directemente en las celdas del Notebook y en algunos puntos además con **matplotlib y Seaborn**.

---

## Índice  

1. [Introducción](#-Introducción)
2. [Estructura del proyecto](#-estructura-del-proyecto)   
3. [Modelado de datos](#-modelado-de-datos)   
4. [Consultas implementadas](#-consultas-implementadas)   
5. [Instrucciones de ejecución](#-instrucciones-de-ejecución)
6. [Conclusiones y aprendizajes](#-conclusiones-y-aprendizajes)  

---

## Estructura del proyecto 

```text
viajes-db/
│
├── notebooks/
│   ├── fuentes/
│   │   ├── usuarios.csv
│   │   ├── destinos.csv
│   │   ├── hoteles.csv
│   │   ├── actividades.csv
│   │   ├── reservas.csv
│   │   └── usuarios_relaciones.csv
│   │  
│   ├── src/
│   │   ├── mongo.py
│   │   ├── neo4j.py
│   │   ├── redis.py
│   │   └── utils.py
│   │
│   ├── Carga de Datos.ipynb
│   ├── Consultas.ipynb
│   ├── Informe Final.ipynb
│   ├── db_connections.py
│   └── constants.py
│
├── python/
│   ├── Dockerfile
│   └── requirements.txt
│
├── generador_datos.py
├── docker-compose.yml
└── README.md

```
## Componenetes Principales 

1. **`Generación de Datos`:** Script Python con Faker que genera datos sintéticos realistas de usuarios, destinos, hoteles, actividades y reservas.
2. **`Procesamiento`:** Módulos especializados (`mongo.py`, `neo4j.py`, `redis.py`) manejan la lógica específica de cada base de datos.
3. **`Carga de Datos`:** Pipeline que lee CSVs, transforma datos y los distribuye según el tipo de base de datos.
4. **`Consultas Integradas`:** Combinación de consultas a las tres bases de datos para obtener información completa.

Dentro de la carpeta **fuentes** se almacen los datos de las distintas entidades que permiten la carga inicial de las distintas bases. Estas fuentes son creadas con el script de **generador_datos.py**.

En **src** se almacenan funciones creadas para modularizar el código realizado.

Luego, existen los notebooks de para la carga de datos y para la ejecución de las consignas del trabajo práctico.

---


## Modelado de Datos 

Para el funcionamiento del sistema se utilizan tres bases de datos NoSql con fines distintos:

| Base de datos | Rol principal | Tipo |
|----------------|---------------|------|
| **MongoDB** | Almacena entidades y registros persistentes | Documental |
| **Neo4j** | Representa relaciones y permite recomendaciones | Grafos |
| **Redis** | Gestiona datos temporales y en caché | Clave-valor |

### Arquitectura Polyglot Persistence

```text
           ┌──────────┐
           │  Usuario │
           └────┬─────┘
                │
      ┌─────────┴─────────┐
      │     JupyterLab    │
      │ (Capa de lógica)  │
      └───┬─────┬─────┬───┘
          │     │     │
     ┌────┘     │     └───────────┐
┌──────────┐  ┌──────────┐  ┌──────────┐
│ MongoDB  │  │  Neo4j   │  │  Redis   │
│ Document │  │ Graph DB │  │ In-memory│
└──────────┘  └──────────┘  └──────────┘

```
**¿En qué consiste esta arquitectura?**

Consiste en utilizar diferentes tipos de bases de datos dentro de una misma aplicación, eligiendo la tecnología más adecuada según la naturaleza de los datos y las operaciones requeridas. En lugar de depender de una única base para resolver todas las necesidades, se adoptó una combinación que aprovecha las fortalezas de cada modelo.

Esta estrategia refleja una tendencia moderna en el desarrollo de sistemas de información, donde la especialización y la interoperabilidad entre tecnologías generan soluciones más eficientes, escalables y cercanas a los escenarios del mundo real.


### MongoDB
Colecciones:
- `usuarios`
``` json
{
  "usuario_id": 1,
  "nombre": "Dario",
  "apellido": "Micheli",
  "email": "email@gmail.com",
  "telefono": "+34843181960"
}
``` 
- `destinos`
``` json
{
  "destino_id": 1,
  "provincia": "Buenos Aires",
  "ciudad": "La Plata",
  "pais": "Argentina",
  "tipo": "cultural",
  "precio_promedio": 50000
}
```
- `actividades`
``` json
{
    "actividad_id": 1,
    "nombre": "Culipatin",
    "tipo": "deportiva",
    "ciudad": "Bariloche",
    "provincia": "Rio Negro",
    "precio": 10000
}
``` 
- `hoteles`
``` json
{
 "hotel_id": 1,
 "nombre": "AMAU",
 "ciudad": "La Plata",
 "provincia": "Buenos Aires",
 "precio": 80000,
 "calificacion": 5,
 "servicios" : ["spa","wifi"]   
}
``` 
- `reservas`
``` json
{
  "reserva_id": 1,
  "usuario_id": 2,
  "destino_id": 1,
  "hotel_id":1,
  "fecha_reserva": "2025-07-01",
  "estado": "Confirmada",
  "precio_total": 90000
}

```

### Neo4j
Nodos:
- `(:Usuario {usuario_id})`
- `(:Destino {destino_id})`

Relaciones:
- `(:Usuario)-[:VISITO]->(:Destino)`
- `(:Usuario)-[:AMIGO_DE]->(:Usuario)`

### Redis
Estructuras:
- `STRING usuario:{usuario_id}:sesion`. Ejemplo:
``` bash
Usuario:15:sesion "activa"
``` 
- `STRING busqueda:{tipo}:{parametro de busqueda}`. Ejemplo:
``` bash
busqueda:destinos:ciudad:La Rioja|tipo:Playa {'destino_id': 17, 'provincia': 'La Rioja', 'ciudad': 'La Rioja', 'pais': 'Argentina', 'tipo': 'Playa', 'precio_promedio': 138236}
```
- `HASH reserva_temp:{reserva_id}`. Ejemplo:
``` bash
reserva_temp:960: {'usuario_id': '18', 'destino_id': '17', 'fecha_reserva': '2024-12-02', 'precio_total': '130068'}
```
---

## Proceso de Carga

1. **Lectura de CSVs**  
   - Función: `utils.lectura_csv()`  
   - Descripción: Carga datos desde la carpeta `fuentes/`.

2. **Inserción en MongoDB**  
   - Función: `insertar_en_mongo()`  
   - Descripción: Crea colecciones y carga documentos en la base de datos.

3. **Creación de Nodos en Neo4j**  
   - Función: `crear_nodos_neo4j()`  
   - Descripción: Genera nodos para **usuarios** y **destinos**.

4. **Creación de Relaciones**  
   - `crear_relaciones_visito()`: Analiza reservas **confirmadas/pagadas**.  
   - `crear_relaciones_usuarios()`: Carga **amistades** y **relaciones familiares**.

5. **Carga en Redis**  
   - `guardar_usuarios_conectados()`: Selecciona 15 usuarios aleatorios.  
   - `carga_masiva_reservas_temporales()`: Guarda reservas **sin estado**.




---

## Consultas Implementadas

| N° | Descripción | Base | 
|----------------|---------------|------|
| **1.A** | Usuarios que visitaron “Bariloche” | Neo4j 
| **1.B** | Amigos de Juan que visitaron algún destino que visitó él | Neo4j 
| **1.C** | Sugerir destinos a un usuario que no haya visitado él ni sus amigos | Neo4j 
| **1.D** | Recomendar destinos basados en viajes de amigos | Neo4j 
| **1.E** | Listar los hoteles en los destinos recomendados del punto D| MongoDB 
| **1.F** | Ver las reservas en proceso | Redis 
| **1.G** | Listar los usuarios conectados actualmente | Redis 
| **1.H** | Mostrar los destinos con precio inferior a $100.000 | Redis y MongoDB 
| **1.I** | Mostrar todos los Hoteles de “Jujuy” | Redis y MongoDB 
| **1.J** | Mostrar la cantidad de hoteles de un destino que guste | MongoDB 
| **1.K** | Mostrar las actividades de “Ushuaia” del tipo “aventura” | Redis y MongoDB 
| **1.L** | reservas concretadas de cada usuario | Redis y MongoDB 
| **2.I** | Destino más visitado | MongoDB 
| **2.II** | Hotel más barato | MongoDB 
| **2.III** | Actividad más popular | MongoDB 
| **3.A** | Incrementar el precio de las actividades de Tucuman en 5% | MongoDB 
| **3.B** | Agregar al hotel id=1 el servicio de SPA | MongoDB 
| **3.C** | Eliminar el destino que desee | MongoDB 
| **3.D** | Eliminar un usuario que desee | MongoDB 
| **3.E** | Eliminar las relaciones AMIGO_DE para un usuario que quiera. | Neo4j


---

## Instrucciones de ejecución

1. Ejecutar **generador_datos.py**: Este script creará los archivos csv en la carpeta /**fuentes**
2. Abrir notebooks/**Carga de Datos.ipynb**
3. Ejecutar todas las celdas en orden
4. Abrir notebooks/**Consultas.ipynb**
5. Ejecutar todas las celdas en orden

---

## Conclusiones y aprendizajes

El desarrollo de este proyecto permitió integrar tres tecnologías NoSQL con propósitos complementarios, demostrando el valor de la arquitectura polyglot persistence. **MongoDB** resultó fundamental para gestionar información estructurada y flexible de usuarios, destinos y reservas; **Neo4j** facilitó representar y consultar relaciones complejas de manera natural, simplificando la generación de recomendaciones basadas en vínculos entre personas y lugares; y **Redis** aportó velocidad y eficiencia para manejar datos temporales como búsquedas, sesiones y reservas en proceso. 

La combinación de estas bases dentro de un mismo entorno de trabajo evidenció cómo cada tecnología resuelve un aspecto diferente del problema: persistencia, conocimiento y tiempo real. En conjunto, el proyecto fortaleció la comprensión sobre cómo elegir la herramienta adecuada según el tipo y comportamiento de los datos, y mostró que la integración inteligente de distintos modelos permite construir soluciones más escalables, dinámicas y cercanas a los escenarios reales del mundo digital actual.

### ¿Por qué MongoDB para Datos Transaccionales?

- **✓ Flexibilidad de esquema:** Los hoteles tienen arrays variables de servicios

```json
{
  "hotel_id": 42,
  "nombre": "Hotel Amau",
  "servicios": ["wifi", "spa", "pileta", "gimnasio"],
  "calificacion": 4
}
```

- **✓Consultas eficientes:** Agregaciones para estadísticas (contador(), contar_documentos())

- **✓Escalabilidad horizontal:** 

- **✓Operaciones CRUD simples:** insert_many, find con filtros y proyecciones


### ¿Por qué Neo4j para Relaciones?
### Recomendación de destinos basada en amigos
Facil para hacer estadisticas entre relaciones:


```python
MATCH (u:Usuario {usuario_id: $id})-[:AMIGO_DE]-(amigo)-[:VISITO]->(d:Destino)
WHERE NOT (u)-[:VISITO]->(d)
RETURN d.ciudad, count(*) AS amigos_visitaron
ORDER BY amigos_visitaron DESC
```


**✓ Modelado intuitivo**: 

**Performance en travesías**: Mucho más rápido que JOINs recursivos en SQL.

**Análisis de red**: Fácil identificar clusters de usuarios y destinos populares


### ¿Por qué Redis para Datos Temporales?


- ✓ **Velocidad:** Operaciones en microsegundos, ideal para sesiones.  
- ✓ **TTL automático:** Reservas temporales expiran sin intervención manual.  
- ✓ **Caché inteligente:** Búsquedas frecuentes se sirven desde memoria.  
- ✓ **Reduce carga:** Menos consultas a MongoDB y Neo4j.

## Decisiones Específicas de Diseño
**Decisiones de Diseño Específicas:**

- **Reservas confirmadas vs temporales:**  
  Las confirmadas van a MongoDB (persistencia) + Neo4j (relación `VISITO`), las temporales solo a Redis (efímeras).  

- **Límites de relaciones:**  
  Máximo 8 amigos y 2 familiares por usuario para evitar explosión de conexiones.  

- **Filtro de fecha en `VISITO`:**  
  Solo reservas pasadas crean la relación (`fecha_reserva <= hoy`).  

- **Servicios de hoteles como array:**  
  Uso de `ast.literal_eval()` para convertir strings en listas reales en MongoDB.  

- **Usuarios conectados aleatorios:**  
  Simulación realista de 15 usuarios online en cualquier momento.
